In [52]:
# import os
# import hashlib
# import ecdsa
# import base58

# # 生成256位随机私钥
# def generate_private_key():
#     return os.urandom(32)

# # 从私钥生成公钥（使用ECDSA算法）
# def private_key_to_public_key(private_key):
#     # 使用secp256k1椭圆曲线生成签名密钥
#     sk = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)
#     pk = sk.get_verifying_key()
#     return pk.to_string()

# # 将公钥转换为比特币地址（使用SHA256和RIPEMD160哈希）
# def public_key_to_address(public_key):
#     # 第一步：对公钥进行SHA256哈希
#     sha256_public_key = hashlib.sha256(public_key).digest()
    
#     # 第二步：对SHA256的结果进行RIPEMD160哈希
#     ripemd160 = hashlib.new('ripemd160', sha256_public_key).digest()
    
#     # 第三步：加上版本字节（对于比特币主网，版本字节是0x00）
#     versioned_payload = b'\x00' + ripemd160
    
#     # 第四步：计算校验和（对版本字节+公钥哈希进行双重SHA256哈希，取前4个字节作为校验和）
#     checksum = hashlib.sha256(hashlib.sha256(versioned_payload).digest()).digest()[:4]
    
#     # 第五步：将校验和附加到版本字节+公钥哈希后
#     address_bytes = versioned_payload + checksum
    
#     # 第六步：使用Base58Check编码最终结果
#     address = base58.b58encode(address_bytes)
    
#     return address.decode('utf-8')

# # 生成比特币账户（返回私钥、公钥和比特币地址）
# def generate_bitcoin_account():
#     # 生成私钥（256位）
#     private_key = generate_private_key()
    
#     # 生成公钥
#     public_key = private_key_to_public_key(private_key)
    
#     # 生成比特币地址
#     bitcoin_address = public_key_to_address(public_key)
    
#     # 返回私钥、公钥和比特币地址
#     return private_key.hex(), public_key.hex(), bitcoin_address

# # 示例用法
# private_key, public_key, bitcoin_address = generate_bitcoin_account()
# print(f"私钥: {private_key}")
# print(f"公钥: {public_key}")
# print(f"比特币地址: {bitcoin_address}")


In [53]:
import struct

# RIPEMD-160 初始值
H0, H1, H2, H3, H4 = 0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476, 0xC3D2E1F0

# RIPEMD-160 常量
K = [0x00000000, 0x5A827999, 0x6ED9EBA1, 0x8F1BBCDC, 0xA953FD4E]
KK = [0x50A28BE6, 0x5C4DD124, 0x6D703EF3, 0x7A6D76E9, 0x00000000]

# 左旋转函数
def left_rotate(x, n):
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# RIPEMD-160 的五个基本运算函数
def f(j, x, y, z):
    if 0 <= j <= 15:
        return x ^ y ^ z
    elif 16 <= j <= 31:
        return (x & y) | (~x & z)
    elif 32 <= j <= 47:
        return (x | ~y) ^ z
    elif 48 <= j <= 63:
        return (x & z) | (y & ~z)
    elif 64 <= j <= 79:
        return x ^ (y | ~z)

# 主处理块
def process_block(block):
    # 处理块数据
    X = list(struct.unpack('<16I', block))

    # 初始化工作寄存器
    A, B, C, D, E = H0, H1, H2, H3, H4
    AA, BB, CC, DD, EE = H0, H1, H2, H3, H4

    # 第一个处理循环
    for j in range(80):
        T = (left_rotate(A + f(j, B, C, D) + X[j % 16] + K[j // 16], (j % 5) + 5) + E) & 0xFFFFFFFF
        A, E, D, C, B = E, D, left_rotate(C, 10), B, T

        T = (left_rotate(AA + f(79 - j, BB, CC, DD) + X[(5 * j + 1) % 16] + KK[j // 16], (j % 5) + 5) + EE) & 0xFFFFFFFF
        AA, EE, DD, CC, BB = EE, DD, left_rotate(CC, 10), BB, T

    # 将两个工作寄存器的值进行合并
    A = (A + H0 + DD) & 0xFFFFFFFF
    B = (B + H1 + EE) & 0xFFFFFFFF
    C = (C + H2 + AA) & 0xFFFFFFFF
    D = (D + H3 + BB) & 0xFFFFFFFF
    E = (E + H4 + CC) & 0xFFFFFFFF

    return A, B, C, D, E

def ripemd160(data):
    # 初始化哈希值
    h0, h1, h2, h3, h4 = H0, H1, H2, H3, H4

    # 填充数据
    data = bytearray(data)
    data_len_bits = len(data) * 8
    data.append(0x80)
    while len(data) % 64 != 56:
        data.append(0)
    data += struct.pack('<Q', data_len_bits)

    # 分块处理
    for i in range(0, len(data), 64):
        h0, h1, h2, h3, h4 = process_block(data[i:i+64])

    # 输出哈希值
    return struct.pack('<5I', h0, h1, h2, h3, h4)


In [54]:
import struct

# SHA-256 常量值
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

# 初始哈希值
H = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

# 右旋操作
def right_rotate(value, shift):
    return ((value >> shift) | (value << (32 - shift))) & 0xffffffff

# SHA-256 压缩函数
def sha256_compress(chunk):
    w = [0] * 64
    w[0:16] = struct.unpack('>16I', chunk)
    
    for i in range(16, 64):
        s0 = right_rotate(w[i - 15], 7) ^ right_rotate(w[i - 15], 18) ^ (w[i - 15] >> 3)
        s1 = right_rotate(w[i - 2], 17) ^ right_rotate(w[i - 2], 19) ^ (w[i - 2] >> 10)
        w[i] = (w[i - 16] + s0 + w[i - 7] + s1) & 0xffffffff

    a, b, c, d, e, f, g, h = H

    for i in range(64):
        S1 = right_rotate(e, 6) ^ right_rotate(e, 11) ^ right_rotate(e, 25)
        ch = (e & f) ^ (~e & g)
        temp1 = (h + S1 + ch + K[i] + w[i]) & 0xffffffff
        S0 = right_rotate(a, 2) ^ right_rotate(a, 13) ^ right_rotate(a, 22)
        maj = (a & b) ^ (a & c) ^ (b & c)
        temp2 = (S0 + maj) & 0xffffffff

        h = g
        g = f
        f = e
        e = (d + temp1) & 0xffffffff
        d = c
        c = b
        b = a
        a = (temp1 + temp2) & 0xffffffff

    H[0] = (H[0] + a) & 0xffffffff
    H[1] = (H[1] + b) & 0xffffffff
    H[2] = (H[2] + c) & 0xffffffff
    H[3] = (H[3] + d) & 0xffffffff
    H[4] = (H[4] + e) & 0xffffffff
    H[5] = (H[5] + f) & 0xffffffff
    H[6] = (H[6] + g) & 0xffffffff
    H[7] = (H[7] + h) & 0xffffffff

def sha256(data):
    data = bytearray(data)
    length = len(data) * 8
    data.append(0x80)

    while (len(data) * 8) % 512 != 448:
        data.append(0)

    data += struct.pack('>Q', length)

    for i in range(0, len(data), 64):
        sha256_compress(data[i:i+64])

    return b''.join(struct.pack('>I', h) for h in H)

message = b"Hello, world!"
hash_result = sha256(message)
print(hash_result.hex())    


315f5bdb76d078c43b8ac0064e4a0164612b1fce77c869345bfc94c75894edd3


In [55]:
import os

# 椭圆曲线参数
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
A = 0
B = 7
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
G = (Gx, Gy)

# 模逆运算
def mod_inverse(a, n):
    t, newt = 0, 1
    r, newr = n, a % n
    while newr != 0:
        quotient = r // newr
        t, newt = newt, t - quotient * newt
        r, newr = newr, r - quotient * newr
    if r > 1:
        raise ValueError("a is not invertible")
    if t < 0:
        t += n
    return t

# 点是否在椭圆曲线上
def is_on_curve(P):
    if P is None:
        return True
    x, y = P
    return (y * y - (x * x * x + A * x + B)) % p == 0

# 椭圆曲线上的点加法
def point_add(P, Q):
    if P is None:  # 无穷远点
        return Q
    if Q is None:
        return P
    if P == Q:  # 自加
        if P[1] == 0:  # 如果 y 为 0，则结果是无穷远点
            return None
        lmbd = (3 * P[0] * P[0] * mod_inverse(2 * P[1], p)) % p
    elif P[0] == Q[0]:  # x 坐标相等，结果为无穷远点
        return None
    else:
        lmbd = ((Q[1] - P[1]) * mod_inverse(Q[0] - P[0], p)) % p
    x = (lmbd * lmbd - P[0] - Q[0]) % p
    y = (lmbd * (P[0] - x) - P[1]) % p
    return (x, y)

# 椭圆曲线点乘法（双倍加法算法）
def point_multiply(k, P):
    R = None  # 无穷远点
    addend = P

    while k > 0:
        if k & 1:  # 如果当前位是 1
            R = point_add(R, addend)
        addend = point_add(addend, addend)  # 点加倍
        k >>= 1
    return R


def ecdsa_sign(message, private_key):
    if isinstance(message, str):
        message = message.encode()  # 如果是字符串，先编码为字节
    
    e = int.from_bytes(sha256(message), 'big')  # 消息的哈希值
    
    while True:
        # 生成一个随机数 k，确保 1 <= k < n
        k = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
        
        # 计算椭圆曲线点 (x1, y1) = k * G
        x1, _ = point_multiply(k, G)
        
        # 计算 r = x1 mod n
        r = x1 % n
        if r == 0:
            continue  # 如果 r 为 0，重新生成 k
        
        # 计算 s = k^(-1) * (e + r * private_key) mod n
        k_inv = mod_inverse(k, n)
        s = (k_inv * (e + r * private_key)) % n
        if s == 0:
            continue  # 如果 s 为 0，重新生成 k
        
        # 返回签名对 (r, s)
        return r, s



def is_on_curve(P):
    if P is None:
        return True
    x, y = P
    return (y * y - (x * x * x + A * x + B)) % p == 0

# 测试点乘
private_key = 0x1E99423A4ED27608A15A2616FC5F1CA99DA7C9A02B3A4FA0AC4914EFB27DA45D
public_key = point_multiply(private_key, G)

assert is_on_curve(public_key), "公钥不在曲线上"
print("点乘验证通过！")


点乘验证通过！


In [56]:
import os

# 从私钥生成公钥
def private_key_to_public_key(private_key):
    # 将私钥转为整数
    priv_key_int = int.from_bytes(private_key, 'big')
    
    # 使用基点倍增生成公钥
    public_key_point = point_multiply(priv_key_int, G)
    
    # 将公钥编码为非压缩格式（0x04 + x + y）
    x, y = public_key_point
    public_key = b'\x04' + x.to_bytes(32, 'big') + y.to_bytes(32, 'big')
    
    return public_key


# Base58 编码实现
def base58_encode(data):
    alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    num = int.from_bytes(data, 'big')
    encode = ''
    
    while num > 0:
        num, mod = divmod(num, 58)
        encode = alphabet[mod] + encode
    
    # 处理前导0
    n_pad = len(data) - len(data.lstrip(b'\0'))
    return '1' * n_pad + encode

# 将公钥转换为比特币地址
def public_key_to_address(public_key):
    # 第一步：对公钥进行 SHA-256 哈希
    sha256_public_key = sha256(public_key)
    
    # 第二步：对 SHA-256 的结果进行 RIPEMD-160 哈希
    ripemd160_hash = ripemd160(sha256_public_key)
    
    # 第三步：加上版本字节（比特币主网版本字节是 0x00）
    versioned_payload = b'\x00' + ripemd160_hash
    
    # 第四步：计算校验和（双重 SHA-256 哈希的前 4 字节）
    checksum = sha256(sha256(versioned_payload))[:4]
    
    # 第五步：将校验和附加到版本字节 + 公钥哈希
    address_bytes = versioned_payload + checksum
    
    # 第六步：Base58 编码最终结果
    address = base58_encode(address_bytes)
    
    return address

# 生成比特币账户（返回私钥、公钥和比特币地址）
def generate_bitcoin_account():
    # 生成256位随机私钥
    private_key = os.urandom(32)
    
    # 生成公钥
    public_key = private_key_to_public_key(private_key)
    
    # 生成比特币地址
    bitcoin_address = public_key_to_address(public_key)
    
    # 返回私钥、公钥和比特币地址
    return private_key.hex(), public_key.hex(), bitcoin_address

# 示例用法
private_key, public_key, bitcoin_address = generate_bitcoin_account()
print(f"私钥: {private_key}")
print(f"公钥: {public_key}")
print(f"比特币地址: {bitcoin_address}")


私钥: 2f9bc37f594118a687155f70a3dc81041b84c2873932e879efd8ae766fa875a2
公钥: 048d17394a7138cd1ed9343a90dc66449ef4df0d273170ca0e1b82e026ef69893d52145dcb3774bb3b94f7d166d836e366305ce9f5c211ba3e01ca7285bf882181
比特币地址: 1FpDWbuy7yts9dRy4hQpoJdDZ45j5kW4Km


In [63]:
import json

# 创建一个交易消息
def create_transaction(from_address, to_address, amount):
    # 创建简单的交易结构
    transaction = {
        "from": from_address,
        "to": to_address,
        "amount": amount
    }
    return json.dumps(transaction)

# 使用私钥对交易进行签名
def sign_transaction(transaction, private_key):
    # 计算交易的哈希 
    tx_hash = sha256(transaction.encode())
    # 使用私钥进行 ECDSA 签名 
    r, s = ecdsa_sign(tx_hash, int(private_key, 16))
    # 将签名转换为可读格式
    signature = f"{r:x}{s:x}"
    return signature

from_address = bitcoin_address
to_address = "12bRukky1ddgzchvZZ6uTXYmgbTfANqpwr"
amount = 0.1

# 创建交易
transaction = create_transaction(from_address, to_address, amount)

# 使用账户A的私钥对交易进行签名
signature = sign_transaction(transaction, private_key)

print("交易信息:", transaction)
print("交易签名:", signature)


交易信息: {"from": "1FpDWbuy7yts9dRy4hQpoJdDZ45j5kW4Km", "to": "12bRukky1ddgzchvZZ6uTXYmgbTfANqpwr", "amount": 0.1}
交易签名: 4319758391f22919b9a1417359559a679fbb39d60f1f0316bb2c2f4e184eb45a51a9493c1b2126800e48e9f1efce0ac9fab937baa61eabb40fdc645bfad821d3
